### Credit Card Payment Default - Amazon SageMaker

For this example, we will follow the example described in this AWS Blog: https://aws.amazon.com/es/blogs/machine-learning/simplify-machine-learning-with-xgboost-and-amazon-sagemaker/

First, we will load some libraries...

In [2]:
import boto3
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sagemaker
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
from sagemaker.serializers import CSVSerializer

We will now define a few variables...

In [4]:
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/xgboost_credit_risk'
role = sagemaker.get_execution_role()
print("Amazon S3 location: s3://{}/{}/".format(bucket,prefix))
print("AWS IAM role:", role)

Amazon S3 location: s3://sagemaker-eu-west-1-889960878219/sagemaker/xgboost_credit_risk/
AWS IAM role: arn:aws:iam::889960878219:role/service-role/AmazonSageMaker-ExecutionRole-20180920T165537


We will use a sample dataset from the University of California Irvine (UCI)'s Machine Learning repository, called: “default of credit card clients” https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients

Let us download the dataset...

In [5]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls

--2021-03-13 22:59:41--  https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5539328 (5.3M) [application/x-httpd-php]
Saving to: ‘default of credit card clients.xls’

default of credit c 100%[===================>]   5.28M  3.79MB/s    in 1.4s    

2021-03-13 22:59:43 (3.79 MB/s) - ‘default of credit card clients.xls’ saved [5539328/5539328]



Now we will load the dataset file into a Pandas dataframe and visualize it...

In [6]:
dataset = pd.read_excel('default of credit card clients.xls')
pd.set_option('display.max_rows', 8)
pd.set_option('display.max_columns', 15)
dataset

,Unnamed: 0,X1,X2,X3,X4,X5,X6,...,X18,X19,X20,X21,X22,X23,Y
0,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,...,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,1,20000,2,2,1,24,2,...,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,-1,...,0,1000,1000,1000,0,2000,1
3,3,90000,2,2,2,34,0,...,1518,1500,1000,1000,1000,5000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29997,29997,150000,1,3,2,43,-1,...,1837,3526,8998,129,0,0,0
29998,29998,30000,1,2,2,37,4,...,0,0,22000,4200,2000,3100,1
29999,29999,80000,1,3,1,41,1,...,85900,3409,1178,1926,52964,1804,1
30000,30000,50000,1,2,1,46,0,...,2078,1800,1430,1000,1000,1000,1


We will now remove the unnamed rows, and split the daframe into Training and Validation datasets, with a 70% and 10% of rows for each...

In [7]:
dataset = dataset.drop('Unnamed: 0', axis=1)
dataset = pd.concat([dataset['Y'], dataset.drop(['Y'], axis=1)], axis=1)

In [8]:
train_data, validation_data, test_data = np.split(dataset.sample(frac=1, random_state=1729), [int(0.7 * len(dataset)), int(0.9 * len(dataset))])
train_data.to_csv('train.csv', header=False, index=False)
validation_data.to_csv('validation.csv', header=False, index=False)

Now, we upload the train and validation datasets to our Amazon S3 storage...

In [9]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')
s3_input_train = TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = TrainingInput(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

We will define our model estimator for the training, in this case by relying on the Amazon SageMaker SDK and pre-defined container image for the XGBoost algorithm...

In [10]:
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'}

sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(containers[boto3.Session().region_name],
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)

Now we run the training job...

In [11]:
xgb.set_hyperparameters(eta=0.1, objective='binary:logistic', num_round=25) 
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

2021-03-13 23:00:18 Starting - Starting the training job...
2021-03-13 23:00:42 Starting - Launching requested ML instancesProfilerReport-1615676417: InProgress
......
2021-03-13 23:01:43 Starting - Preparing the instances for training......
2021-03-13 23:02:44 Downloading - Downloading input data...
2021-03-13 23:03:06 Training - Downloading the training image..Arguments: train
[2021-03-13:23:03:27:INFO] Running standalone xgboost training.
[2021-03-13:23:03:27:INFO] File size need to be processed in the node: 2.32mb. Available memory size in the node: 8440.08mb
[2021-03-13:23:03:27:INFO] Determined delimiter of CSV input is ','
[23:03:27] S3DistributionType set as FullyReplicated
[23:03:27] 21000x23 matrix with 483000 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2021-03-13:23:03:27:INFO] Determined delimiter of CSV input is ','
[23:03:27] S3DistributionType set as FullyReplicated
[23:03:27] 6000x23 matrix with 138000 entries loaded from /opt/ml/

Our model is now trained, with the resulting artifacts stored in Amazon S3, so we are ready for deploying an Amazon SageMaker Endnpoint for performing real-time predictions...

In [12]:
xgb_predictor = xgb.deploy(
    initial_instance_count = 1,
    instance_type = 'ml.m4.xlarge',
    serializer = CSVSerializer())

-------------!

Let us test our model with a few predictions using our testing dataset...

In [13]:
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(test_data.to_numpy()[:,1:])
predictions

array([0.10145303, 0.29316297, 0.70782304, ..., 0.15233986, 0.38610485,
       0.10773233])

The resulting array is showing the probability for each customer to be in default for their credit card payments.

-----